<a href="https://colab.research.google.com/github/hiropppe/takeGo/blob/develop/colab/train_rl_policy_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (113 kB/s)
Selecting previously unselected package tree.
(Reading database ... 131015 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
!tree /content/drive/MyDrive/Develop/alphago

/content/drive/MyDrive/Develop/alphago
├── test
│   └── policy
│       └── feature_planes.h5
├── test_data
│   ├── rl_policy
│   │   └── initial_weights.hdf5
│   └── sl_policy
│       └── feature_planes.h5
└── test_training
    ├── rl_policy
    │   ├── metadata.json
    │   ├── weights.00000.hdf5
    │   ├── weights.00001.hdf5
    │   └── weights.00002.hdf5
    └── sl_policy
        ├── logs
        │   ├── train
        │   │   ├── events.out.tfevents.1713665721.e94ad11756a1.38497.0.v2
        │   │   └── events.out.tfevents.1713666228.e94ad11756a1.38497.2.v2
        │   └── validation
        │       ├── events.out.tfevents.1713665884.e94ad11756a1.38497.1.v2
        │       └── events.out.tfevents.1713666390.e94ad11756a1.38497.3.v2
        ├── metadata.json
        ├── models
        │   ├── assets
        │   ├── fingerprint.pb
        │   ├── keras_metadata.pb
        │   ├── saved_model.pb
        │   └── variables
        │       ├── variables.data-00000-of-00001
        │      

In [4]:
!mkdir -p /content/drive/MyDrive/Develop/alphago/test_data/rl_policy/
!mkdir -p /content/drive/MyDrive/Develop/alphago/test_training/rl_policy/

!curl -L https://github.com/hiropppe/takeGo/raw/develop/params/policy/kihuu.hdf5 -o /content/drive/MyDrive/Develop/alphago/test_data/rl_policy/initial_weights.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 29.6M  100 29.6M    0     0  8199k      0  0:00:03  0:00:03 --:--:-- 9139k


In [5]:
!pip install cython
!pip install sgf
!pip install watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00


In [6]:
%load_ext watermark
%watermark -u -d -p cython,tensorflow,h5py,numpy,pandas,matplotlib,sklearn

Last updated: 2024-04-22

cython    : 3.0.10
tensorflow: 2.15.0
h5py      : 3.9.0
numpy     : 1.25.2
pandas    : 2.0.3
matplotlib: 3.7.1
sklearn   : 1.2.2



In [7]:
%load_ext Cython

In [8]:
!git clone https://github.com/hiropppe/takeGo.git
%cd takeGo/
!python setup.py build_ext -i

Cloning into 'takeGo'...
remote: Enumerating objects: 2679, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 2679 (delta 208), reused 258 (delta 129), pack-reused 2311
Receiving objects: 100% (2679/2679), 293.03 MiB | 19.45 MiB/s, done.
Resolving deltas: 100% (1951/1951), done.
Updating files: 100% (152/152), done.
/content/takeGo
Compiling bamboo/board.pyx because it changed.
Compiling bamboo/zobrist_hash.pyx because it changed.
Compiling bamboo/pattern.pyx because it changed.
Compiling bamboo/printer.pyx because it changed.
Compiling bamboo/parseboard.pyx because it changed.
Compiling bamboo/tree_search.pyx because it changed.
Compiling bamboo/seki.pyx because it changed.
Compiling bamboo/nakade.pyx because it changed.
Compiling bamboo/local_pattern.pyx because it changed.
Compiling bamboo/policy_feature.pyx because it changed.
Compiling bamboo/rollout_preprocess.pyx because it changed.
Compiling bamboo/sgf_util.py

In [9]:
import json
import numpy as np
import os
import time

import tensorflow as tf

from shutil import copyfile

from tensorflow import keras
from tensorflow.keras import backend as K

from bamboo.models.keras_dcnn_policy import KerasPolicy, cnn_policy

from bamboo.self_play_game import run_n_games

In [10]:
def log_loss(y_true, y_pred):
    '''Keras 'loss' function for the REINFORCE algorithm, where y_true is the action that was
    taken, and updates with the negative gradient will make that action more likely. We use the
    negative gradient because keras expects training data to minimize a loss function.
    '''
    return -y_true * K.log(K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon()))


def start_training(args):

    ZEROTH_FILE = "weights.00000.hdf5"

    if args.resume:
        if not os.path.exists(os.path.join(args.out_directory, "metadata.json")):
            raise ValueError("Cannot resume without existing output directory")

    if not os.path.exists(args.out_directory):
        if args.verbose:
            print("creating output directory {}".format(args.out_directory))
        os.makedirs(args.out_directory)

    if not args.resume:
        # make a copy of weights file, "weights.00000.hdf5" in the output directory
        copyfile(args.initial_weights, os.path.join(args.out_directory, ZEROTH_FILE))
        if args.verbose:
            print("copied {} to {}".format(args.initial_weights,
                                           os.path.join(args.out_directory, ZEROTH_FILE)))
        learner_weights = ZEROTH_FILE
    else:
        # if resuming, we expect initial_weights to be just a
        # "weights.#####.hdf5" file, not a full path
        args.initial_weights = os.path.join(args.out_directory,
                                            os.path.basename(args.initial_weights))
        if not os.path.exists(args.initial_weights):
            raise ValueError("Cannot resume; weights {} do not exist".format(args.initial_weights))
        elif args.verbose:
            print("Resuming with weights {}".format(args.initial_weights))
        learner_weights = os.path.basename(args.initial_weights)

    learner_model = cnn_policy()
    learner_model.load_weights(args.initial_weights)
    learner_policy = KerasPolicy(learner_model)

    opponent_model = cnn_policy()
    opponent_policy = KerasPolicy(opponent_model)

    if args.verbose:
        print("created player and opponent with temperature {}".format(args.policy_temp))

    if not args.resume:
        metadata = {
            "init_weights": args.initial_weights,
            "learning_rate": args.learning_rate,
            "temperature": args.policy_temp,
            "game_batch": args.game_batch,
            "opponents": [ZEROTH_FILE],  # which weights from which to sample an opponent each batch
            "win_ratio": {}  # map from player to tuple of (opponent, win ratio) Useful for
                             # validating in lieu of 'accuracy/loss'
        }
    else:
        with open(os.path.join(args.out_directory, "metadata.json"), "r") as f:
            metadata = json.load(f)

    # Append args of current run to history of full command args.
    metadata["cmd_line_args"] = metadata.get("cmd_line_args", [])
    metadata["cmd_line_args"].append(vars(args))

    def save_metadata():
        with open(os.path.join(args.out_directory, "metadata.json"), "w") as f:
            json.dump(metadata, f, sort_keys=True, indent=2)

    optimizer = keras.optimizers.SGD(learning_rate=args.learning_rate)
    learner_model.compile(loss=log_loss, optimizer=optimizer)

    for i_iter in range(1, args.iterations + 1):
        # Randomly choose opponent from pool (possibly self), and playing
        # game_batch games against them.
        opp_weights = np.random.choice(metadata["opponents"])
        opp_path = os.path.join(args.out_directory, opp_weights)

        # Load new weights into opponent's network, but keep the same opponent object.
        opponent_model.load_weights(opp_path)
        if args.verbose:
            print("Batch {}\tsampled opponent is {}".format(i_iter, opp_weights))

        state_tensors, move_tensors, learner_won, win_ratio = run_n_games(learner_policy, opponent_policy)
        print(f'winning ratio: {win_ratio:.2f}')
        # Train on each game's results, setting the learning rate negative to 'unlearn' positions from
        # games where the learner lost.
        for (st_tensor, mv_tensor, won) in zip(state_tensors, move_tensors, learner_won):
            #print(len(st_tensor), len(mv_tensor), st_tensor[0].shape, mv_tensor[0].shape, won)
            # optimizer.lr = K.abs(optimizer.lr) * (+1 if won else -1)
            K.set_value(optimizer.lr, abs(args.learning_rate) * (+1 if won else -1))
            learner_model.train_on_batch(np.concatenate(st_tensor, axis=0),
                                 np.concatenate(mv_tensor, axis=0))

        metadata["win_ratio"][learner_weights] = (opp_weights, win_ratio)

        # Save intermediate models.
        if i_iter % args.record_every == 0:
            learner_weights = "weights.{:05d}.hdf5".format(i_iter)
            learner_model.save_weights(os.path.join(args.out_directory, learner_weights))

        # Add player to batch of oppenents once in a while.
        if i_iter % args.save_every == 0:
            metadata["opponents"].append(learner_weights)

        save_metadata()

In [11]:
!rm -rf /content/drive/MyDrive/Develop/alphago/test_training/rl_policy/
!mkdir -p /content/drive/MyDrive/Develop/alphago/test_training/rl_policy/

args = {
    'initial_weights': '/content/drive/MyDrive/Develop/alphago/test_data/rl_policy/initial_weights.hdf5',
    'out_directory': '/content/drive/MyDrive/Develop/alphago/test_training/rl_policy/',
    'learning_rate': 0.001,
    'policy_temp': 0.67,
    'save_every': 1,
    'record_every': 1,
    'game_batch': 2,
    'move_limit': 500,
    'iterations': 2,
    'resume': False,
    'verbose': True,  # Turn on verbose mode
}

from types import SimpleNamespace
start_args = SimpleNamespace(**args)
start_training(start_args)

copied /content/drive/MyDrive/Develop/alphago/test_data/rl_policy/initial_weights.hdf5 to /content/drive/MyDrive/Develop/alphago/test_training/rl_policy/weights.00000.hdf5
created player and opponent with temperature 0.67
Batch 1	sampled opponent is weights.00000.hdf5
Score(1): -25.5
Score(0): -34.5
Score(2): -35.5
winning ratio: 0.33
Batch 2	sampled opponent is weights.00000.hdf5
Score(1): 6.5
Score(0): 2.5
Score(2): -29.5
winning ratio: 0.33
